In [108]:
import pandas as pd
from tqdm import tqdm

In [110]:
df_name = 'beer'
df_orig = pd.read_csv(f'data/{df_name}/{df_name}.csv')
df_dirty = pd.read_csv(f'data/{df_name}/{df_name}_all_columns_20.csv')

In [111]:
numerical_columns = df_dirty.select_dtypes(include='number').columns.to_list()
categorical_columns = [col for col in df_dirty.columns if col not in numerical_columns]


In [112]:
v_counts = {col: None for col in df_dirty.columns}
quantiles = {col: None for col in df_dirty.columns}
for col in df_dirty.columns:
    v_counts[col]=df_dirty.value_counts(col)
    quantiles[col] = v_counts[col].quantile(.20)

In [113]:
n_split = {col: {} for col in df_dirty.columns}
for col in df_dirty.columns:
    for val in v_counts[col].index:
        n_split[col][val] = v_counts[col][val]//quantiles[col] + (v_counts[col][val]%quantiles[col]>0)

In [114]:
v_counts

{'beer_name': beer_name
 flying_monkey_amber_ale         2
 rockets_red_ale                 2
 100_meter_ale                   1
 prohibition_ale                 1
 pike_pale_heirloom_amber_ale    1
                                ..
 fat_tire_amber_ale              1
 fattener_14                     1
 fiddler_s_green_amber           1
 fin_de_si_cle                   1
 zz_amber_ale                    1
 Length: 798, dtype: int64,
 'brew_factory_name': brew_factory_name
 cigar_city_brewing                6
 rogue_ales                        6
 peak_organic_brewing_company      5
 world_brews                       5
 rock_bottom_restaurant_brewery    4
                                  ..
 galena_brewing_company            1
 garrison_brewing_company          1
 geneva_lake_brewing_company       1
 georgetown_brewing_company        1
 yukon_brewing_company             1
 Length: 656, dtype: int64,
 'style': style
 american_amber_red_ale    800
 dtype: int64,
 'abv': abv
 5.0     81
 5

In [ ]:
counts = n_split.copy()

In [104]:
df_orig_strat = df_orig.copy()
df_dirty_strat = df_dirty.copy()

for idx, row in tqdm(df_orig.iterrows(), total=len(df_orig)):
    for col in df_dirty.columns:
        if col in numerical_columns:
            continue
        quantile = quantiles[col]
        val = row[col]
        count = counts[col][val]
        df_orig_strat.loc[idx, col] = f'{val}_{int(count)}'
        if not pd.isna(val):
            df_dirty_strat.loc[idx, col] = f'{val}_{int(count)}'
            counts[col][val]-=1
        if counts[col][val] == 0:
            counts[col][val] = n_split[col][val]

In [106]:
df_dirty_strat.to_csv(f'data/main-exp-batch/new_datasets_dirty/{df_name}_all_columns_20_stratified.csv', index=False)
df_orig_strat.to_csv(f'data/main-exp-batch/new_datasets/{df_name}_stratified.csv', index=False)
